In [52]:
# Code in this cell makes plots appear an appropriate size and resolution in the browser window
%matplotlib widget
%config InlineBackend.figure_format = 'svg'

import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (11, 6)

In [53]:
from firedrake import *
from numpy import linspace

In [54]:
from firedrake.adjoint import *
continue_annotation()

True

In [55]:
n = 100
mesh = PeriodicIntervalMesh(n, length=2)
x = SpatialCoordinate(mesh)[0]
nu = Constant(1e-2)

In [56]:
V = FunctionSpace(mesh, "Lagrange", 2)
u_n1 = Function(V, name="u^{n+1}")
u_n = Function(V, name="u^{n}")
v = TestFunction(V)
u_init = Function(V).interpolate(sin(2*pi*x))
u_n.assign(u_init)
dt = 1.0 / n

In [57]:
F = (((u_n1 - u_n)/dt) * v +
     u_n1 * u_n1.dx(0) * v + 
     nu*u_n1.dx(0)*v.dx(0))*dx
problem = NonlinearVariationalProblem(F, u_n1)
solver = NonlinearVariationalSolver(problem)

In [58]:
results = [Function(u_n)]
t = 0
t_end = 0.1
for t in ProgressBar("Time step").iter(linspace(0.0, t_end, int(t_end/dt))):
    solver.solve()
    u_n.assign(u_n1)
    results.append(Function(u_n))

Time step ▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣ 10/10 [0:00:00]


In [59]:
J = assemble(u_n*u_n*dx)

In [60]:
pause_annotation()

In [61]:
tape=get_working_tape()

In [62]:
tape.visualise("tape.pdf")

In [63]:
import base64
from IPython.display import IFrame
with open("tape.pdf", "rb") as pdf:
    content = pdf.read()

# encode PDF
base64_pdf = base64.b64encode(content).decode("utf-8")

# display encoded PDF
IFrame(f"data:application/pdf;base64,{base64_pdf}", width=1000, height=500)

In [51]:
tape.clear_tape()